# red-blue_channel_compare

Notebook to plot two mass channels for each given FOV:
+ One will be colored <span style="color:red;text-shadow:0px 0px 2px white">__red__</span>
+ One will be colored <span style="color:cyan;text-shadow:0px 0px 2px black">__blue__</span>

Points of overlap appear <span style="color:white;text-shadow:0px 0px 2px black">__white__</span>  
Images are saved as `.png` files within a directory structure headed at input `save_path`

Accepted format for images: IonPath's __MIBITiff__ format from MIBI/O or MIBITracker

Please update the __input parameters__ as needed

## load headers

In [75]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pathlib
import glob
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## input parameters

In [76]:
data_path = pathlib.Path('/Volumes/DISK_IMG/')
save_path = pathlib.Path('/Users/adam_kagel/Desktop/RBcomp/')

# Foreground/Target channel (-1 for final processing stage)
blue_channel = 115
b_processing_stage = -1

# Background/Donor channel
red_channel = 197
r_processing_stage = 0

# pixels w/ non zero values -> 1.0 (shows all overlaps equally)
binary = True

# channels rescaled w/ max value -> 1.0 (recommended)
normalized = True

# brighten image (recommended)
brighten = True

# Anonymized by default currently
# TODO: Read panel if anonymize is false, and use target names
#anonymize_targets = True
#panel_path = data_path.joinpath('Panel54_MelanomaCohort_2.csv')

slide_foldernames = [
    '191204_CohortSlide9_S14-47859-A2-S14-20352-A8',
    #'191205_CohortSlide7_S146936A2_G1471023A1_run',
    #'191206_CohortSlide11_TMA443_run',
    #'191211_CohortSlide10_S158066A1_SP153393_G1675892_run'
]

# leave empty to use full slide name
slide_names = [
    'S9',
    #'S7',
    #'S11',
    #'S10'
]

process_folder_prefix = ''
process_folder_suffix = '_TIFF'

raw_tiff_folder_name = 'bg_none'
proccessed_tiff_folder_name = 'bg_au_050_ta_020'

# process 'prefix' is PointN
process_suffix = '_RowNumber0_Depth_Profile0'
isobar_suffix = '-MassCorrected'
denoise_suffix = '-Filtered'


## plot images

In [78]:
# prevents memory overload from a billion figures :)
plt.close('all')

plt.ioff()
plt.style.use('dark_background')

def make_fig(tiffs,slide,point):
    
    # blue channel (technically cyan)
    b_image = tiff.read(str(tiffs[b_processing_stage]))
    b_im = b_image[blue_channel]
    dim = np.zeros(b_im.shape)
    B = np.stack((dim,b_im/2,b_im/2), axis=2)
    
    # red channel (should have same shape as blue channel)
    r_image = tiff.read(str(tiffs[r_processing_stage]))
    r_im = r_image[red_channel]
    R = np.stack((r_im,dim,dim), axis=2)
    
    # move to [0,1] scale
    R, B = R/256., B/256.
    
    # plotting options
    if binary:
        R, B = np.ceil(R), np.ceil(B)
    else:
        if normalized:
            R, B = R/float(np.amax(R)), B/float(np.amax(B))
        if brighten:
            R, B = np.power(R,1/2), np.power(B,1/2)
    
    # make plot
    plt.imshow(R+B)
    plt.gca().set_frame_on(False)
    plt.gca().xaxis.set_visible(False)
    plt.gca().yaxis.set_visible(False)
    plt.tight_layout()
    plt.gcf().set_figheight(1024/300)
    plt.gcf().set_figwidth(1024/300)
    
    # create directories for saving
    if not save_path.is_dir():
        save_path.mkdir()
    comp_dir_name = f'{slide}-{point}'
    comp_dir_path = save_path.joinpath(comp_dir_name)
    if not comp_dir_path.is_dir():
        comp_dir_path.mkdir()
    
    # save as .png for lossless compression
    # TODO: better file naming convention
    plt.savefig(str(comp_dir_path.joinpath(f'{blue_channel}_v_{red_channel}_binary_{binary}.png')), bbox_inches='tight', dpi=300, pad_inches=0.0)
    
# main loop
for slide_n, slide in enumerate(slide_foldernames):
    tiffpath = data_path.joinpath(slide).joinpath(process_folder_prefix+slide+process_folder_suffix)
    rawpath = tiffpath.joinpath(raw_tiff_folder_name)
    
    # search for different points in raw tiff file directory
    # not a robust approach
    # TODO: User inputs desired points/toggles all points in slide
    for rawfilename in glob.glob(f'{str(rawpath)}/*{process_suffix}.tiff'):
        pointNum = int(re.search(r'\d+',pathlib.Path(rawfilename).name).group())
        
        proc_tiffpath = tiffpath.joinpath(proccessed_tiff_folder_name)
        
        tiffs = [
            rawpath.joinpath(rawfilename),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}{denoise_suffix}.tiff')
        ]
        
        make_fig(tiffs,slide_names[slide_n],pointNum)    
            
        #break #debug
    #break #debug